<a href="https://colab.research.google.com/github/bbbbbin2/2025_BigData/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# 사고 유형별 교통사고
df_type = pd.read_excel("/content/사고유형별_교통사고.xls")

# 위반 유형별 교통사고
df_violation = pd.read_excel("/content/위반유형별_교통사고.xls")

# 연도별 교통사고
df_year = pd.read_excel("/content/교통사고통계(연도별).xlsx")

# 기상 상태별 교통사고
df_weather = pd.read_excel("/content/교통사고통계_(기상상태별).xlsx")

# 연령대별 교통사고
df_age = pd.read_excel("/content/연령대별_교통사고.xls")

# 월별 교통사고
df_month = pd.read_excel("/content/교통사고통계_월별.xlsx")


# 지역별 교통사고
df_region = pd.read_excel("/content/지역별_교통사고.xls")

# 화물차 교통사고 다발지역
df_region_cargo = pd.read_csv("/content/화물차.csv", encoding='cp949')

# 음주운전(지역)
df_alcohol1 = pd.read_excel("/content/음주운전(지역)교통사고통계.xlsx")
df_alcohol2 = pd.read_excel("/content/교통사고통계(음주 3년간).xlsx")
df_alcohol3 = pd.read_excel("/content/교통사고통계_음주운전(연령).xlsx")





df_type.head(), df_violation.head(), df_year.head(), df_age.head(), df_weather.head(), df_month.head(), df_region.head(), df_region_cargo.head(), df_alcohol1.head(), df_alcohol2.head(), df_alcohol3.head()


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

(    교통사고지표 현황 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5  \
 0        사고유형       발생건수       대형사고     중대한 사고        NaN       사망자수   
 1         NaN        NaN        NaN      여객(건)      화물(건)        NaN   
 2         횡단중      65944          1          2        843       2248   
 3       차도통행중      19789          1          0        256        750   
 4  길가장자리구역통행중      10211          0          0         89        170   
 
   Unnamed: 6  
 0     치사율(%)  
 1        NaN  
 2       3.41  
 3       3.79  
 4       1.66  ,
   교통사고지표 현황 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6
 0      법규위반       발생건수       대형사고     중대한 사고        NaN       사망자수     치사율(%)
 1       NaN        NaN        NaN      여객(건)      화물(건)        NaN        NaN
 2     중앙선침범      39034         13         17        269        899        2.3
 3      신호위반     117785         20         14        960       1152       0.98
 4   안전거리미확보     108772         31          6        857        259 

In [13]:
import pandas as pd

# 1. 엑셀 로드
def load_excel(path, header=0):
    return pd.read_excel(path, header=header)


# 2. 2줄 헤더 병합 해제
def clean_two_header(df):
    df = df.copy()
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index(drop=True)
    return df


# 3. 완전 비어있는 행/열 제거
def drop_empty(df):
    return df.dropna(how="all").dropna(axis=1, how="all")


# 4. 숫자형 문자열 → int 변환
def to_int(df, cols):
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[c] = (
                df[c]
                .astype(str)
                .str.replace(",", "")
                .replace("-", 0)
                .astype(int)
            )
    return df


# 5. 특정 값 제거 (합계, 알수없음 등)
def drop_values(df, col, values=("합계", "알수없음")):
    df = df.copy()
    for v in values:
        df = df[df[col] != v]
    return df


# 6. 특정 지표 필터링
def filter_eq(df, col, value):
    return df[df[col] == value].copy()


# 7. 연도 컬럼 자동 탐색
def get_year_cols(df):
    return [c for c in df.columns if str(c).isdigit()]


# 8. wide → long
def melt_table(df, id_cols, value_cols,
               var_name="연도", value_name="값"):
    return df.melt(
        id_vars=id_cols,
        value_vars=value_cols,
        var_name=var_name,
        value_name=value_name
    )


# 9. 지역명 통일
def normalize_region(df, col):
    replace_dict = {
        "전라남도": "전남", "전라북도": "전북",
        "경상남도": "경남", "경상북도": "경북",
        "충청남도": "충남", "충청북도": "충북",
        "강원도": "강원",
        "부산광역시": "부산", "대구광역시": "대구",
        "광주광역시": "광주", "인천광역시": "인천",
        "대전광역시": "대전", "울산광역시": "울산",
        "제주특별자치도": "제주",
        "세종특별자치시": "세종",
        "서울특별시": "서울"
    }
    df = df.copy()
    df[col] = df[col].replace(replace_dict, regex=True)
    return df

def find_main_col(df, candidates):
    """
    후보 컬럼명 중 실제 df에 존재하는 첫 번째 컬럼 반환
    """
    for c in candidates:
        if c in df.columns:
            return c
    raise KeyError(f"대표 컬럼을 찾을 수 없습니다: {candidates}")



In [12]:
df_violation = (
    load_excel("/content/위반유형별_교통사고.xls", header=1)
    .pipe(drop_empty)
)

violation_col = find_main_col(
    df_violation,
    ["위반유형", "위반유형별", "구분"]
)

df_violation = drop_values(df_violation, violation_col)


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NameError: name 'find_main_col' is not defined

In [14]:
df_year_raw = load_excel("/content/교통사고통계(연도별).xlsx")

year_cols = get_year_cols(df_year_raw)

df_year_long = (
    df_year_raw
    .pipe(lambda x: x[
        (x["분석지표"].isin(["사고건수", "부상자수", "사망자수"])) &
        (x["연도"].isin(["전체", "1일평균"]))
    ])
    .pipe(melt_table,
          id_cols=["분석지표", "연도"],
          value_cols=year_cols,
          var_name="년도",
          value_name="값")
)

df_year_long["년도"] = df_year_long["년도"].astype(int)
df_year_long["지표"] = df_year_long["분석지표"] + "_" + df_year_long["연도"]

df_year = (
    df_year_long
    .pivot(index="년도", columns="지표", values="값")
    .reset_index()
)


In [15]:
# 최근 3년 음주운전 시간대별 사망자
df_raw = load_excel("/content/교통사고통계(음주 3년간).xlsx", header=None)

year_row = df_raw.iloc[0]
time_row = df_raw.iloc[1]
df = df_raw.iloc[2:].reset_index(drop=True)

df.columns = [
    y if y in ["월", "연도"] else f"{y}_{t}"
    for y, t in zip(year_row, time_row)
]

df_death_long = (
    df.pipe(filter_eq, "월", "합계")
      .pipe(filter_eq, "연도", "사망[명]")
)

time_cols = [c for c in df_death_long.columns if "202" in c]

df_death_long = df_death_long[time_cols].T.reset_index()
df_death_long.columns = ["연도_시간대", "사망자수"]
df_death_long["시간대"] = df_death_long["연도_시간대"].str.split("_").str[1]

df_death_long = to_int(df_death_long, ["사망자수"])

df_death = (
    df_death_long
    .groupby("시간대", as_index=False)["사망자수"]
    .sum()
)

df_death["비율"] = df_death["사망자수"] / df_death["사망자수"].sum() * 100


/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [16]:
# 2024년 음주운전 사망사고 연령대
df = clean_two_header(df_alcohol3)

df_death_age = (
    df.pipe(filter_eq, "연도", "사망[명]")
      .pipe(drop_values, "가해운전자(1당) 연령대")
      [["가해운전자(1당) 연령대", "합계"]]
)

df_death_age = to_int(df_death_age, ["합계"])
df_death_age = df_death_age.sort_values("합계", ascending=False)


In [8]:
# 지역 / 경력 / 연령 데이터에서 “알수없음” 제거
for col in ["연령대별", "운전경력", "시도"]:
    if col in df_age.columns:
        df_age = drop_values(df_age, col)
    if col in df_region.columns:
        df_region = drop_values(df_region, col)


In [9]:
# 화물차 사고 지역 통합 + TOP10
df_region_cargo = normalize_region(df_region_cargo, "지점명")

df_cargo_grouped = (
    df_region_cargo
    .groupby("지점명", as_index=False)
    .agg({
        "사고건수": "sum",
        "사상자수": "sum",
        "사망자수": "sum",
        "중상자수": "sum",
        "경상자수": "sum",
        "부상신고자수": "sum",
        "경도": "mean",
        "위도": "mean"
    })
)

df_cargo_top10 = df_cargo_grouped.sort_values("사고건수", ascending=False).head(10)
df_cargo_dead_top = df_cargo_grouped.sort_values("사망자수", ascending=False).head(10)


In [10]:
# 연령대 × 사고원인 히트맵 데이터
df = clean_two_header(df_alcohol3)

df_accident = (
    df.pipe(filter_eq, "연도", "사고[건]")
      .pipe(drop_values, "가해운전자(1당) 연령대")
)

cause_cols = [
    "중앙선 침범", "신호위반", "안전거리 미확보",
    "안전운전 의무 불이행", "교차로 통행방법 위반",
    "보행자 보호의무 위반", "기타"
]

df_heat = df_accident[["가해운전자(1당) 연령대"] + cause_cols]
df_heat = to_int(df_heat, cause_cols)
df_heat = df_heat.set_index("가해운전자(1당) 연령대")
